# County data growth rate and 7day average

Pull data from https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/<br>
Merge cases and deaths, then compute growth rate of cases and deaths as well as 7day moving averages for each county<br>
Outputs combined_usafacts.csv

Read both csvs and inner join

In [1]:
import numpy as np
import pandas as pd
import time
import os

start_time = time.time()

cases_df = pd.read_csv("https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv")
cases_df = cases_df.melt(id_vars=["countyFIPS", "County Name", "State", "stateFIPS"],
        var_name="Date",
        value_name="Cases")

deaths_df = pd.read_csv("https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_deaths_usafacts.csv")
deaths_df = deaths_df.melt(id_vars=["countyFIPS", "County Name", "State", "stateFIPS"],
        var_name="Date",
        value_name="Deaths")
deaths_df = deaths_df[['countyFIPS', 'Date', 'Deaths']]

main_df = pd.merge(cases_df, deaths_df, "inner", on=['countyFIPS', 'Date'])
main_df = main_df.rename(columns={'County Name': 'County_Name'})
main_df

,countyFIPS,County_Name,State,stateFIPS,Date,Cases,Deaths
0,0,Statewide Unallocated,AL,1,1/22/20,0,0
1,0,Statewide Unallocated,AL,1,1/22/20,0,0
2,0,Statewide Unallocated,AL,1,1/22/20,0,0
3,0,Statewide Unallocated,AL,1,1/22/20,0,0
4,0,Statewide Unallocated,AL,1,1/22/20,0,0
...,...,...,...,...,...,...,...
937065,56037,Sweetwater County,WY,56,7/5/20,106,0
937066,56039,Teton County,WY,56,7/5/20,140,1
937067,56041,Uinta County,WY,56,7/5/20,184,0
937068,56043,Washakie County,WY,56,7/5/20,39,5


Get list of FIPS and drop 0

In [2]:
fips = main_df.countyFIPS.unique()
fips = fips[1:]
fips

array([ 1001,  1003,  1005, ..., 56041, 56043, 56045], dtype=int64)

Compute cases growth and death growth rates and 7day average for each county

In [3]:
# Suppress warnings
pd.options.mode.chained_assignment = None

for i in range(0, len(fips)):
    temp_df = main_df[main_df.countyFIPS == fips[i]]
    temp_df['dxdt'] = temp_df['Cases'].diff().fillna(0)
    temp_df['rate'] = temp_df['Cases'].pct_change().fillna(0)
    temp_df['rate7day'] = temp_df['rate'].rolling(window=7).mean().fillna(0)
    temp_df['dydt'] = temp_df['Deaths'].diff().fillna(0)
    temp_df['rate_deaths'] = temp_df['Deaths'].pct_change().fillna(0)
    temp_df['rate_deaths7day'] = temp_df['rate_deaths'].rolling(window=7).mean().fillna(0)
    temp_df = temp_df.replace(np.inf, 0)
    if i == 0:
        new_df = temp_df
    else:
        new_df = pd.concat([new_df, temp_df])
new_df

,countyFIPS,County_Name,State,stateFIPS,Date,Cases,Deaths,dxdt,rate,rate7day,dydt,rate_deaths,rate_deaths7day
2500,1001,Autauga County,AL,1,1/22/20,0,0,0.0,0.0,0.000000,0.0,0.0,0.0
8145,1001,Autauga County,AL,1,1/23/20,0,0,0.0,0.0,0.000000,0.0,0.0,0.0
13790,1001,Autauga County,AL,1,1/24/20,0,0,0.0,0.0,0.000000,0.0,0.0,0.0
19435,1001,Autauga County,AL,1,1/25/20,0,0,0.0,0.0,0.000000,0.0,0.0,0.0
25080,1001,Autauga County,AL,1,1/26/20,0,0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
914489,56045,Weston County,WY,56,7/1/20,2,0,0.0,0.0,0.142857,0.0,0.0,0.0
920134,56045,Weston County,WY,56,7/2/20,2,0,0.0,0.0,0.142857,0.0,0.0,0.0
925779,56045,Weston County,WY,56,7/3/20,2,0,0.0,0.0,0.142857,0.0,0.0,0.0
931424,56045,Weston County,WY,56,7/4/20,2,0,0.0,0.0,0.142857,0.0,0.0,0.0


Export to csv

In [4]:
new_df.to_csv(os.path.join("output", "combined_usafacts.csv"), index=False)

Total Running time (s)

In [5]:
time.time() - start_time

145.19500017166138